In [ ]:
%run -i ../python/common.py
%run -i ../python/ln_preamble.py

# SLS Lecture 8 :  Writing some simple assembly programs



Spend some time writing some very simple assembly programs and learn to use the debugger so that we have enough skills to explore how things work.    We will be repeat various things in more detail in future lectures.

- Write `popcnt` in assemble code 
  - use gdb to play with the popcnt program
- Write a simple `add` in assembly code
  - use gdb to play with the add program
    - using the cpu as a glorified calculator 
      - first pass at CPU support for "numbers" 
- What happens if we let our programs continue
  - how do we successfully "halt/end" our execution 
    - `int3` trap 
       - tells OS to return control to debugger   
    - more generally how can we make a Kernel/System Call
  - revisit `add` programs adding exits  
    - `int3`
    - `exit` syscall
- Implicitly use our shell, editor, Make and Git knowledge to do the above


## Writing a `popcnt` assembly program

- Write a one instruction assembly program 
  1. first using .byte 
  2. using intel assembly instruction  
- Use gdb to explore how this instruction works
  - learn to use gdb to set register values
  - and how to execute and re-execute an instruction

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/popcnt_bb.S", 
    lang="gas", 
    title="<b>CODE: asm - The 'popcnt' assembly program",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown('''
Here is a fully commented version of the same code.
'''))
display(Markdown(FileCodeBox(
    file="../src/popcnt.S", 
    lang="gas", 
    title="<b>CODE: asm - The commented 'popcnt' assembly program",
    h="100%", 
    w="107em"
)))

In [ ]:
display(showET("Editor"))

We can use the `.byte` directive to set the values in memory to anything we like 
eg.

``` gas
     .byte 0xF3, 0x48, 0x0F, 0xB8, 0xD8  // popcnt rax,rbx
```

But of course the real value s that we could have also simply written

``` gas
       popcnt rax, rbx          // ok the single intel opcode (instruction) that makes up
                                // our program
```


In [ ]:
display(showBT("Build"))

In [ ]:
display(Markdown(FileCodeBox(
    file="popcnt_build.sh", 
    lang="shell", 
    title="<b>NOTES: on building popcnt", 
    h="15em", 
    w="40em")))

In [ ]:
display(showDT("Debugger"))

In [ ]:
display(Markdown(FileCodeBox(
    file="popcnt_gdb.txt", 
    lang="shell", 
    title="", 
    h="4em", 
    w="100%")))

## Writing an `add` assembly program

- re-enforce the steps to creating and debugging an assembly program
  - begin to explore CPU support for working with "numbers"
    - cpu as a calculator

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/add.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="107em"
)))

In [ ]:
display(showET("Editor"))

- Lets work with the `add` instruction in a similar way that we did with `popcnt`
- explore the results of adding with binary, hex, unsigned and signed values
- explore overflow

- then make the program a little more complex:
``` gas
  movabs rbx, 0xdeadbeefdeadbeef
  mov    rax, 1
  add    rax, rbx
```

- lets use some more cool features of the intel instruction set
``` gas
  rdrand rbx                
  mov    rax, 1
  add    rax, rbx
  popcnt rbx, rax
```
- lets get a brief glimpse at how to use memory locations for the value
``` gas
        .intel_syntax noprefix
        .data
x:       .quad 142
y:       .quad 4200
sum:     .quad

        .text
        .global _start
_start:
        mov rax, QWORD PTR x
        add rax, QWORD PTR y
        mov QWORD PTR sum, rax
        int3
```
  
  
- try replacing add with `imul`, `and`, `or`, `xor`



In [ ]:
display(showBT())

In [ ]:
display(Markdown(FileCodeBox(
    file="add_build.sh", 
    lang="shell", 
    title="<b>NOTES: on building add", 
    h="15em", 
    w="40em")))

In [ ]:
display(showDT())

In [ ]:
display(Markdown(FileCodeBox(
    file="add_gdb.txt", 
    lang="shell", 
    title="", 
    h="4em", 
    w="100%")))

## Ending / Exiting our Program/Process

- What happens if we run our programs outside of the debugger?
  - why does this happen?

In [ ]:
display(showET())

In [ ]:
display(showBT())

In [ ]:
display(showDT())

### How can we avoid this

1. TRAP: Use an instruction that tells the OS to 
    - stop the process and give control back to the debuggger
    - if no debugger is running just kill process and signal shell
        - Instruction: `int3`: 
        - Opcode: `0xCC` 
        - Description: `Interrupt 3 — trap to debugger`
2. Call OS Kernel Exit Process call
    - This is an example of calling an OS Kernel call to have the kernel do something for your process
    - We will look at this more but for the moment here is what is necessary to call `exit`
       - pass return value to Kernel 
       - exit/terminate process

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/exit_bb_bb.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown(
'''
A commented version that avoids "magic" numbers. 
'''    +
    
    FileCodeBox(
    file="../src/exit.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="107em"
)))